#### New to Plotly?
Plotly's Python library is free and open source! [Get started](https://plot.ly/python/getting-started/) by dowloading the client and [reading the primer](https://plot.ly/python/getting-started/).
<br>You can set up Plotly to work in [online](https://plot.ly/python/getting-started/#initialization-for-online-plotting) or [offline](https://plot.ly/python/getting-started/#initialization-for-offline-plotting) mode, or in [jupyter notebooks](https://plot.ly/python/getting-started/#start-plotting-online).
<br>We also have a quick-reference [cheatsheet](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf) (new!) to help you get started!

#### Simple Dropdown Widget

In [1]:
import pandas as pd
import datetime

from ipywidgets import widgets 
from IPython.display import display, clear_output, Image

import plotly.plotly as py
from plotly.graph_objs import *
import plotly
from plotly.widgets import GraphWidget

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning:

The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning:

IPython.utils.traitlets has moved to a top-level traitlets package.



<IPython.core.display.Javascript object>

In [2]:
contour_plot = GraphWidget('https://plot.ly/~bronsolo/63')
line_plot = GraphWidget('https://plot.ly/~chris/2150')

display(contour_plot)
display(line_plot)

In [ ]:
contour_fig = py.get_figure('https://plot.ly/~bronsolo/63')

In [ ]:
def march(x0, y0, x1, y1):
    '''
    Return the closest path of integer coordinates 
    between (x0, y0) and (x1, y1)
    '''
    if abs(x1-x0) > abs(y1-y0):    
        if x1>x0:
            x = range(int(x0), int(x1)+1)
        else:
            x = range(int(x0), int(x1)+1, -1)
        y = []
        tanth = (y1-y0)/(x1-x0)
        for xi in x:
            y.append(round(y0+(xi-x0)*tanth))
    else:
        if y1>y0:
            y = range(int(y0), int(y1)+1)
        else:
            y = range(int(y0), int(y1)+1, -1)
        x = []
        tanth = (x1-x0)/(y1-y0)
        for yi in y:
            x.append(round(x0+(yi-y0)*tanth))

    return (x, y)

class Responder(object):
    '''
    Stateful object that stores and computes the 
    elevation and distance data of the 
    line plot. The 'click' method is executed
    on `click` events of the contour map.
    '''
    def __init__(self, data):
        self.clicks = 0
        self.x = []
        self.y = []
        self.xp = []
        self.yp = []
        self.z = []
        self.d = []
        self.data = data

    def append_point(self, point):
        xp = point['pointNumber'][1]
        yp = point['pointNumber'][0]
        self.xp.append(xp)
        self.yp.append(yp)

        if 'x' in self.data[0]:
            x = self.data[0]['x'][xp]
        else:
            x = xp
        if 'y' in self.data[0]:
            y = self.data[0]['y'][xp]
        else:
            y = yp

        self.x.append(x)
        self.y.append(y)
        self.z.append(point['z'])
    
        if len(self.x) == 1:
            self.d.append(0)
        else:
            self.d.append(math.sqrt((y-self.y[-2])**2+(x-self.x[-2])**2))
        
        
    def click(self, contour_widget, click_obj):
        point = click_obj[0]
        if self.clicks==0 or self.clicks > 5:
            self.__init__(self.data)
            self.append_point(point)
        else:
            
            (xpath, ypath) = march(self.xp[-1], self.yp[-1],
                                   point['pointNumber'][1], point['pointNumber'][0])
    
            d = []
            z = []
            for i in range(1, len(xpath)):
                xpi = xpath[i]
                ypi = ypath[i]

                if 'x' in self.data[0]:
                    xi = self.data[0]['x'][xpi]
                else:
                    xi = xpi
                if 'y' in self.data[0]:
                    yi = self.data[0]['y'][ypi]
                else:
                    yi = ypi
                
                self.d.append(self.d[-1]+math.sqrt((yi-self.y[-1])**2+(xi-self.x[-1])**2))
                self.z.append(self.data[0]['z'][int(ypi)][int(xpi)])
                self.x.append(xi)
                self.y.append(yi)
                self.xp.append(xpi)
                self.yp.append(ypi)
            

        self.clicks+=1
        
        line_plot.restyle({'x': [self.d], 'y': [self.z]})

    
r = Responder(contour_fig['data'])

In [ ]:
contour_plot.on_click(r.click)

In [ ]:
from IPython.display import display, clear_output

def message_handler(widget, msg):
    clear_output()
    print widget._graph_url
    display(msg)

display(contour_plot)
contour_plot.on_click(message_handler)
g.on_hover(message_handler)
g.on_zoom(message_handler)

In [ ]:
print g._graph_url

#### Reference
See https://plot.ly/python/ for more information and examples.

In [ ]:
from IPython.display import display, HTML

display(HTML('<link href="//fonts.googleapis.com/css?family=Open+Sans:600,400,300,200|Inconsolata|Ubuntu+Mono:400,700" rel="stylesheet" type="text/css" />'))
display(HTML('<link rel="stylesheet" type="text/css" href="http://help.plot.ly/documentation/all_static/css/ipython-notebook-custom.css">'))

! pip install git+https://github.com/plotly/publisher.git --upgrade
    
import publisher
publisher.publish(
    'dropdown.ipynb', 'python/dropdown-widget/', 'IPython Widgets | plotly',
    'How to make a dropdown widgets with Plotly',
    title = 'Dropdown Widget in Plotly',
    name = 'Dropdown Widget in Plotly',
    has_thumbnail='true', thumbnail='thumbnail/dropdown-widget.jpg', 
    language='python', page_type='example_index', 
    display_as='chart_events', order=21)